In [1]:
import os
print(os.getcwd())

C:\Users\shravya\Desktop\Network Security Project\NetworkSecurity


In [3]:
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

# Load ratings and movie titles
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'])
movies = pd.read_csv('ml-100k/u.item', sep='|', encoding='latin-1', 
                     names=['movie_id', 'title', 'release_date', 'video_release', 'url'] + [f'genre_{i}' for i in range(19)])

# Prepare data for Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings[['user_id', 'movie_id', 'rating']], reader)

# Verify data
print("Ratings DataFrame:")
print(ratings.head())
print("\nMovies DataFrame:")
print(movies[['movie_id', 'title']].head())

Ratings DataFrame:
   user_id  movie_id  rating  timestamp
0      196       242       3  881250949
1      186       302       3  891717742
2       22       377       1  878887116
3      244        51       2  880606923
4      166       346       1  886397596

Movies DataFrame:
   movie_id              title
0         1   Toy Story (1995)
1         2   GoldenEye (1995)
2         3  Four Rooms (1995)
3         4  Get Shorty (1995)
4         5     Copycat (1995)


In [5]:
# Split data into train and test sets
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Train SVD model
model = SVD(n_factors=100, random_state=42)
model.fit(trainset)

# Evaluate model
predictions = model.test(testset)
rmse = accuracy.rmse(predictions)
print(f"RMSE: {rmse}")

RMSE: 0.9352
RMSE: 0.935171451026933


In [7]:
def get_top_n_recommendations(user_id, n=10):
    all_movie_ids = ratings['movie_id'].unique()
    predictions = [model.predict(user_id, movie_id) for movie_id in all_movie_ids]
    predictions.sort(key=lambda x: x.est, reverse=True)
    top_n = [(pred.iid, pred.est) for pred in predictions[:n]]
    return top_n

# Test recommendations for user ID 1
user_id = 1
recommendations = get_top_n_recommendations(user_id)
print(f"Top 10 recommendations for user {user_id}:")
for movie_id, predicted_rating in recommendations:
    title = movies[movies['movie_id'] == movie_id]['title'].values[0]
    print(f"Movie: {title}, Predicted Rating: {predicted_rating:.2f}")

Top 10 recommendations for user 1:
Movie: Star Wars (1977), Predicted Rating: 5.00
Movie: Empire Strikes Back, The (1980), Predicted Rating: 4.97
Movie: Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963), Predicted Rating: 4.94
Movie: One Flew Over the Cuckoo's Nest (1975), Predicted Rating: 4.94
Movie: Rear Window (1954), Predicted Rating: 4.93
Movie: Hoop Dreams (1994), Predicted Rating: 4.92
Movie: Citizen Kane (1941), Predicted Rating: 4.75
Movie: Wrong Trousers, The (1993), Predicted Rating: 4.73
Movie: Wallace & Gromit: The Best of Aardman Animation (1996), Predicted Rating: 4.72
Movie: 12 Angry Men (1957), Predicted Rating: 4.71


In [9]:
# Create a dictionary of movie IDs to titles for JavaScript
movie_titles = movies.set_index('movie_id')['title'].to_dict()

# Function to get recommendations as a list of dictionaries
def get_recommendations_for_frontend(user_id):
    recommendations = get_top_n_recommendations(user_id)
    return [{'movie_id': int(movie_id), 'title': movie_titles[movie_id], 'predicted_rating': round(pred_rating, 2)} 
            for movie_id, pred_rating in recommendations]

In [1]:
import ipywidgets as widgets
from IPython.display import display

# Create dropdown for user IDs
user_ids = list(range(1, 944))
dropdown = widgets.Dropdown(options=user_ids, description='User ID:', style={'description_width': 'initial'})

# Create output area
output = widgets.Output()

# Function to display recommendations
def show_recommendations(change):
    with output:
        output.clear_output()
        user_id = change['new']
        recommendations = get_top_n_recommendations(user_id)
        print(f"Top 10 recommendations for user {user_id}:")
        for movie_id, predicted_rating in recommendations:
            title = movies[movies['movie_id'] == movie_id]['title'].values[0]
            print(f"Movie: {title}, Predicted Rating: {predicted_rating:.2f}")

# Link dropdown to function
dropdown.observe(show_recommendations, names='value')

# Display the interface
display(dropdown)
display(output)

Dropdown(description='User ID:', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 2…

Output()